In [2]:
from langchain.agents import initialize_agent, AgentType, Tool
from langchain.memory import ConversationBufferMemory
from langgraph.graph import StateGraph
from langchain.chat_models import ChatOpenAI
from langchain_gigachat.chat_models import GigaChat
from langchain.schema import HumanMessage, SystemMessage, Document, AIMessage
from langchain.vectorstores import FAISS
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.chains import RetrievalQA
from langchain_community.tools import DuckDuckGoSearchRun
from langgraph.types import Command
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import StateGraph, END, MessagesState, START
from typing import TypedDict, Optional, Literal, List, Dict
from typing_extensions import TypedDict

import datetime
import warnings
import os
import re

In [3]:
warnings.filterwarnings("ignore")

In [4]:
os.environ["GIGACHAT_CREDENTIALS"] = "Yjc1YWZhNTItMzYwYS00NmU4LTk4YjctZjU4YzAwMDIyMGJmOjJhNWE3YmVhLTAyYzctNGJhNy05NWE3LWEzY2YwNGQzYzZiNw=="

In [5]:
with open("../sources/code_base.md", "r") as f:
    code_base_template = f.read()
with open("../sources/unit_test_base.md", "r") as f:
    unit_test_template = f.read()

In [25]:
llm = GigaChat(
    model="GigaChat-2-Max",
    verify_ssl_certs=False,
    profanity_check=False,
    streaming=True,
    max_tokens=8192,
    temperature=0.3,
    repetition_penalty=1.01,
    timeout=60
)

In [26]:
coder_agent_prompt = """Ты - Senior Python Developer. Твоя задача - написать web-приложение для агентской системы, которая выполняет следующую задачу: {task}.
Для этого воспользуйся шаблоном: {code_template}. В этом шаблоне класс AgentSystem отвечает за логику работы системы, 
она должна быть прописана в методе run. Обертка вокруг этой логиги описана в FastAPI-приложении app. 
Логика должна запускаться после обращения к endpoint run_agent. Также необходимо прописать протоколы взаимодействия в pydantic классах AgentRequest и AgentResponse.
"""

In [33]:
tester_agent_prompt = """Ты - Senior Python Developer. Твоя задача - написать unit-тесты для кода приложения {code}, которое выполняет следующую задачу: {task}.
Для этого воспользуйся шаблоном: {unit_test_template}. Важно, чтобы интеграции были замоканы.
"""

In [34]:
tools = []
coder_agent = create_react_agent(
    llm,
    tools,
    checkpointer=MemorySaver()
)

tester_agent = create_react_agent(
    llm,
    tools,
    checkpointer=MemorySaver()
)

In [28]:
config = {"configurable": {"thread_id": "thread_id2"}}

In [29]:
task = """Реализовать агента, который может складывать два числа."""

In [35]:
with open("../sources/test_code.md", "r") as f:
    test_code = f.read()

In [30]:
code_msg = coder_agent_prompt.format(
    code_template=code_base_template,
    task=task
)
code_task = {
    "messages": [HumanMessage(content=code_msg)]
}

In [37]:
test_msg = tester_agent_prompt.format(
    unit_test_template=unit_test_template,
    task=task,
    code=test_code
)
test_task = {
    "messages": [HumanMessage(content=test_msg)]
}

In [41]:
test_config = {"configurable": {"thread_id": "thread_id4"}}

In [42]:
tester_result = tester_agent.invoke(test_task, config=test_config)

In [43]:
print(tester_result["messages"][1].content)

Для корректного тестирования приложения на основе FastAPI и Pydantic, нужно учесть несколько моментов:

1. Проверить успешное выполнение агента.
2. Проверить обработку ошибок (например, когда агент возвращает ошибку).
3. Проверить корректность обработки входных данных.

Вот пример реализации unit-тестов для данного приложения:

```python
import pytest
from unittest.mock import patch, MagicMock
from fastapi import HTTPException
from app.main import run_agent, AgentRequest, AgentResponse, AgentSystem

# Тестирование успешного выполнения агента
@pytest.mark.asyncio
async def test_run_agent_success():
    # Arrange
    request_data = {"num1": 5.0, "num2": 3.0}
    request = AgentRequest(**request_data)
    expected_result = {"status": "success", "result": 8.0}

    with patch.object(AgentSystem, "run", return_value=expected_result) as mock_agent_run:
        # Act
        response = await run_agent(request)

        # Assert
        assert isinstance(response, AgentResponse)
        assert

In [31]:
result = coder_agent.invoke(task, config=config)

In [24]:
print(result["messages"][1].content)

Вот как можно реализовать агента, который складывает два числа, используя предоставленный шаблон:

```python
# app/main.py
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel

app = FastAPI()

# Реализация агента, который складывает два числа
class AgentSystem:
    def run(self, input_data: dict) -> dict:
        # Проверяем, что в input_data есть оба числа
        if "num1" not in input_data or "num2" not in input_data:
            raise HTTPException(status_code=400, detail="Оба числа должны быть указаны")
        
        # Складываем числа
        result = input_data["num1"] + input_data["num2"]
        
        return {"status": "success", "result": result}

agent = AgentSystem()

# Протокол запроса к агенту
class AgentRequest(BaseModel):
    num1: float
    num2: float

# Протокол ответа от агента
class AgentResponse(BaseModel):
    status: str
    result: float

# Обработчик запроса к агенту
@app.post("/run_agent", response_model=AgentResponse)
async def ru